In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

# Kalman Filter Class (Now Tracking Temperature & Rate of Change)
class KalmanFilter:
    def __init__(self, process_variance, measurement_variance):
        # State Transition Matrix (Tracks Temperature & Rate of Change)
        self.A = np.array([[1, 1],  # x1 (temperature) evolves by adding x2 (velocity)
                           [0, 1]]) # x2 (velocity) remains constant

        # Measurement Matrix (We only observe temperature, not velocity)
        self.H = np.array([[1, 0]])

        # Process Noise Covariance
        self.Q = np.array([[process_variance, 0],
                           [0, process_variance]])

        # Measurement Noise Covariance
        self.R = np.array([[measurement_variance]])

        # Initial State Estimate [temperature, rate of change]
        self.x_hat = np.array([20.0, 0.1])  # Start with 20°C and 0.1°C per step

        # Initial Error Covariance
        self.P = np.eye(2) * 1  # Initial uncertainty

    def predict(self):
        """ Predict next state and update uncertainty """
        self.x_hat = self.A @ self.x_hat  # State prediction
        self.P = self.A @ self.P @ self.A.T + self.Q  # Uncertainty prediction

    def update(self, measurement):
        """ Update the estimate with a new measurement """
        K = self.P @ self.H.T @ np.linalg.inv(self.H @ self.P @ self.H.T + self.R)  # Kalman Gain
        self.x_hat = self.x_hat + K @ (measurement - self.H @ self.x_hat)  # State update
        self.P = (np.eye(2) - K @ self.H) @ self.P  # Update error covariance

    def get_rate_of_change(self):
        """ Return the estimated rate of change (velocity) """
        return self.x_hat[1]  # x2 (velocity)


# Function to simulate the pull-based system using the improved Kalman Filter
def pull_based_system_sink_kalman(z_temp, time_steps, threshold, process_variance, measurement_variance):
    transmission_times = []  # Time steps when the node transmits
    sink_predictions = []  # (Time, Predicted x1 at pull time)
    actual_values_sent = []  # Store the actual z(t) values at transmission times
    reconstructed_values = []  # Reconstructed temperature using rate of change

    # Initialize Kalman filter
    kalman_filter = KalmanFilter(process_variance, measurement_variance)

    last_transmission_time = 0  # Last time a measurement was received
    last_predicted_temp = kalman_filter.x_hat[0]  # Initial temperature
    last_rate_of_change = kalman_filter.get_rate_of_change()  # Initial rate of change

    for t in range(1, time_steps):
        delta_t = t - last_transmission_time  # Time since last transmission

        # **Prediction Step**
        kalman_filter.predict()
        predicted_value = kalman_filter.x_hat[0]  # Predicted Temperature
        rate_of_change = kalman_filter.get_rate_of_change()  # Predicted Rate of Change

        # **Threshold Check**: Pull the node if |Rate of Change * Δt| exceeds threshold
        if (abs(rate_of_change) * delta_t > threshold) or (abs(min_rate_of_change) * delta_t > threshold):
            # The sink pulls the node and receives a new measurement z(t)
            measured_value = z_temp[t]

            # **Update the Kalman filter with the received measurement**
            kalman_filter.update(measured_value)

            # Store transmission times and predicted values at pull time
            transmission_times.append(t)
            sink_predictions.append((t, predicted_value))
            actual_values_sent.append((t, measured_value))

            # **Reconstruction** using the rate of change
            # Use the last predicted temperature and rate of change to estimate the reconstructed value
            reconstructed_temp = last_predicted_temp + last_rate_of_change * delta_t
            reconstructed_values.append((t, reconstructed_temp))

            # Update for the next loop
            last_transmission_time = t  # Update transmission time
            last_predicted_temp = predicted_value  # Update the predicted temperature
            last_rate_of_change = rate_of_change  # Update the rate of change

    return transmission_times, sink_predictions, actual_values_sent, reconstructed_values


# Generate a realistic temperature signal
def generate_realistic_temperature_signal(total_time, time_resolution, T_avg, T_period, noise_amplitude):
    t = np.arange(0, total_time, time_resolution)
    temperature_variation = 2 * np.sin(2 * np.pi * t / (7 * 24))  # Weekly cycle
    omega = 2 * np.pi / T_period
    T_dynamic = temperature_variation * np.sin(omega * t) + T_avg
    noise = np.random.normal(0, noise_amplitude, len(t))
    T_realistic = T_dynamic + noise
    return t, T_realistic


# Simulation Parameters
total_time = 1000  
time_resolution = 1  
T_avg = 20  
T_period = 100  
noise_amplitude = 0.05  

t, T_realistic = generate_realistic_temperature_signal(total_time, time_resolution, T_avg, T_period, noise_amplitude)

# System Parameters
threshold = 0.5  # Threshold for pulling node (Rate of Change * Delta T)
process_variance = 0.01  
measurement_variance = 0.1  
min_rate_of_change = 0.02

# Run the pull-based system simulation (sink-side processing) using Kalman Filter
transmission_times, sink_predictions, actual_values_sent, reconstructed_values = pull_based_system_sink_kalman(
    T_realistic, total_time, threshold, process_variance, measurement_variance
)

# Extract values for plotting
if sink_predictions:
    sink_pred_times, sink_pred_values = zip(*sink_predictions)
else:
    sink_pred_times, sink_pred_values = [], []

if actual_values_sent:
    sent_times, sent_values = zip(*actual_values_sent)
else:
    sent_times, sent_values = [], []

if reconstructed_values:
    reconstructed_times, reconstructed_values = zip(*reconstructed_values)
else:
    reconstructed_times, reconstructed_values = [], []

# **Spline Fitting** (Cubic Spline)
spline = CubicSpline(sent_times, sent_values, bc_type='natural')

# Generate fitted values using spline for the full time range
spline_fitted_values = spline(t)

# **Plot the Results**
plt.figure(figsize=(12, 6))

# Plot actual temperature z(t)
plt.plot(t, T_realistic, label="Actual Temperature Measurements (z(t))", color='green', linestyle='-', alpha=0.7)

# Plot spline-fitted reconstruction of actual data sent to sink
plt.plot(t, spline_fitted_values, label="Spline Fitted Reconstruction of Sent Data", color='blue', linestyle='-', linewidth=2)

# Plot sink predictions at pull times (marked with 'x')
plt.scatter(sink_pred_times, sink_pred_values, color='orange', marker='x', label="Sink Predicted Values (Kalman Filter)")

# Plot actual values sent to sink
plt.scatter(sent_times, sent_values, color='red', marker='o', label="Actual Sent Values (z(t))")

plt.xlabel("Time Step")
plt.ylabel("Temperature (°C)")
plt.title("Spline Fitting Reconstruction vs Actual Temperature Measurements")
plt.legend()
plt.grid(True)
plt.show()

